# Notes #
1. Data preparation
    + Should we normalize the dataset? If yes, then why?
    + Basic steps
        - Load data
        - Check for missing values(*No missing values in this case*)
        - Convert categorical fields to numerical(All categorical fields are represented  
        as digits)
        - Check class distribution
            + Filtering the dataset with the first two classes.
            + The split should then be roughly 50/50.
        - Normalize data if necessary


**Rough notes**  
1.

---

**Data preparation**

In [2]:
# Data preparation #

# Load CSV using Pandas
import pandas as pd
import numpy as np
# filename = 'adult.data1.csv'
# names = []
data = pd.read_csv("covtype.data.csv", delimiter=",", header=None, index_col=False)

data = data[data[54].isin([1,2])]

Y = data.iloc[:, -1]
X = X = data.iloc[:,:-1]

# Normalize the X values #
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

# feature_list = names[:14]
# # Test train split #
# X = data.loc[:, feature_list]
# Y = data[['target']]

# # Split the dataset into test and train datasets

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

X.shape


(495141, 54)

---

**Keras Model**

In [3]:
# Keras #

### Logistic regression using DNN ###
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

np.random.seed(7)

# Defining model #

model = Sequential()
model.add(Dense(1,input_dim=54,activation='sigmoid'))

# Compile the model #

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model #

model.fit(x_train, y_train, epochs=5000, batch_size=100)

# Evaluate the model #

scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/5000
198056/198056 [==============================] - 177s 896us/step - loss: -0.3623 - acc: 0.2600
Epoch 2/5000
198056/198056 [==============================] - 6s 30us/step - loss: -2.0739 - acc: 0.3041
Epoch 3/5000
198056/198056 [==============================] - 5s 27us/step - loss: -3.6365 - acc: 0.3213
Epoch 4/5000
198056/198056 [==============================] - 6s 33us/step - loss: -4.9172 - acc: 0.3604
Epoch 5/5000
198056/198056 [==============================] - 7s 36us/step - loss: -6.0687 - acc: 0.3810
Epoch 6/5000
198056/198056 [==============================] - 7s 35us/step - loss: -7.0782 - acc: 0.3988
Epoch 7/5000
198056/198056 [==============================] - 6s 32us/step - loss: -7.8766 - acc: 0.4158
Epoch 8/5000
198056/198056 [==============================] - 6s 33us/step - loss: -8.4710 - acc: 0.4265
Epoch 9/5000
198056/198056 [==============================] - 6s 29us/step - loss: -8.8938 - acc: 0.4289
Epoch 10/5000
198056/198056 [=======================

KeyboardInterrupt: 

---

**Scikit model**

In [20]:
# Scikit learn #

# Import and create an instance of your model(Logistic regression)

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

# Train your model using the training dataset

logisticRegr.fit(x_train,y_train)

# Predict the output 

predictions = logisticRegr.predict(x_test)
print(predictions)

score = logisticRegr.score(x_test,y_test)
print(score)


[2 2 1 ... 2 1 2]
0.7756837268795126


---

**Kfold validation**

In [ ]:
# Validation #
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

seed = 7
numpy.random.seed(seed)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X, Y):
    # create model
    model = Sequential()
    model.add(Dense(1,input_dim=14,activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X.iloc[train], Y.iloc[train], epochs=500, batch_size=100, verbose=0)
    # evaluate the model
    scores = model.evaluate(X.iloc[test], Y.iloc[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores))) 


---

**ROC Curve and Confusion matrix**

In [ ]:
# Prediction #
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
# y_pred = (y_pred>0.5)
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


# Creating the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

---

**Classification Report**

In [ ]:
from sklearn.metrics import classification_report

target_names = ['<=50K', '>50']
print(classification_report(y_test, y_pred, target_names=target_names))

---

# Pending #
1. See if you can fine tune keras model(75% acc) to match scikit model(78% acc)
2. Exploratory analysis of the dataset
3. Latex explanations
4. **Cross check what's wrong with ROC  
and Confusion matrix**